In [1]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()

+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+



In [2]:
help(env)

Help on Environment in module kaggle_environments.core object:

class Environment(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, specification={}, configuration={}, steps=[], agents={}, interpreter=None, renderer=None, html_renderer=None, debug=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  clone(self)
 |      Returns:
 |          Environment: A copy of the current environment.
 |  
 |  play(self, agents=[], **kwargs)
 |      Renders a visual representation of the environment and allows interactive action selection.
 |      
 |      Args:
 |          **kwargs (dict): Args directly passed into render().  Mode is fixed to ipython.
 |      
 |      Returns:
 |          None: prints directly to an IPython notebook
 |  
 |  render(self, **kwargs)
 |      Renders a visual representation of the current state of the environment.
 |      
 |      Args:
 |          mode (str): html, ipython, ansi, human (default)
 |          **kwargs (d

In [3]:
import env_configurations
import games_configurations
env_name = 'connectx'
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)

print(obs_space)
print(action_space)

Box(7, 6, 3)
Discrete(7)


In [4]:
trainer = env.train([None, "random"])

In [5]:
import dqnagent
import yaml
from runner import Runner
from kaggle_env import KaggleEnv
import tensorflow as tf
import numpy as np

gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.2)



sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.2)))
with open('configs/connectx_ppo.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    runner.sess = sess
    
player = runner.create_player()
#player.restore('nn/best_connectx')
player.restore('nn/last_connectx_ppoep=10001.0rew=0.42')
def ppo_agent(observation, configuration):
    #self.agent.get_masked_action_values([obs],)
    mask = np.array(observation.board[0:7]) == 0
    obs = KaggleEnv.build_obs(True, True, observation)
    action = player.get_masked_action([obs], [mask], True)
    if np.sum(observation['board']) == 0:
        return np.random.randint(7)
    if observation.board[action] != 0:
        print('wrong step')
        print(mask)
        print(action)
        for i in range(7):
            if observation.board[i] == 0:
                return i
    else:
        return int(action)
    
def ppo_agent2(observation, configuration):
    mask = np.array(observation.board[0:7]) == 0
    action = player.get_masked_action([KaggleEnv.build_obs(False, True,observation)], [mask], True)
    if observation.board[action] != 0:
        print('wrong step')
        print(mask)
        for i in range(7):
            if observation.board[i] == 0:
                return i
    else:
        return int(action)


using masks for action
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/last_connectx_ppoep=10001.0rew=0.42


In [6]:
#env.reset()
# Play as the first agent against default "random" agent.
env.run([ppo_agent, "negamax"])
#env.run([ "negamax", ppo_agent2])
env.render(mode="ipython", width=500, height=450)

[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[ -5.6802635   2.726512   -4.771046   19.096529   -2.666414   -4.8550134
  -11.730506 ]]
[[ -1.1846194   7.48632    -2.9454224   8.333259   -2.9731908  -1.8415993
  -11.255339 ]]
[[ 4.0204554  10.26177    -3.5188086  -7.3238745   1.307585   -0.01864021
  -6.646135  ]]
[[ 18.00475      0.26411247   4.5118313  -15.609457    -1.6373322
   -0.6042389   -5.5541387 ]]
[[-9.028265   -2.260898   11.126365   -0.41849554 -3.840647   -6.193764
  -8.728091  ]]
[[ -6.4158134  -2.8402562  10.685699   -5.8513722  -1.0617702  -4.8054605
  -12.681989 ]]
[[ -3.1138747   5.5025053 -16.294226   -3.3459623   2.6356518  -0.7689039
  -11.862925 ]]
[[ -1.7649834   2.2230387 -13.898665   -1.8374528   1.0529954  -1.1840068
   -9.5773   ]]
[[ -1.6997225 -17.759768  -15.113312   18.012342    5.283108   -7.1850476
  -11.985211 ]]


In [7]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / sum(r[0] + r[1] for r in rewards)
def mean_reward2(rewards):
    return 1.0 - sum(r[0] for r in rewards) / sum(r[0] + r[1] for r in rewards)


print("My Agent vs agent2:", mean_reward(evaluate("connectx", [ppo_agent, ppo_agent2], num_episodes=100)))

print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [ppo_agent, "random"], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward2(evaluate("connectx", ["random", ppo_agent], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [ppo_agent2, "random"], num_episodes=100)))
print("My Agent vs Random Agent:", mean_reward2(evaluate("connectx", ["random", ppo_agent2], num_episodes=100)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, my_agent], num_episodes=1000)))
#print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent2, my_agent2], num_episodes=1000)))
#print("My Agent vs Negamax Agent:", mean_reward2(evaluate("connectx", [ "negamax", ppo_agent], num_episodes=100)))
print("My Agent vs Negamax Agent:", mean_reward2(evaluate("connectx", [ "negamax", ppo_agent2], num_episodes=100)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [ppo_agent, "negamax", ], num_episodes=100)))
#print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [ppo_agent2, "negamax", ], num_episodes=100)))





[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[ -6.2004       0.65974045   0.7913815   18.833784    -5.383792
   -4.548825   -10.175612  ]]
[[-7.3466206   1.6244636   4.1721807   6.50288    -6.2807155  -0.81039494
  -6.9203773 ]]
[[ -3.8469706   5.871923    2.019808    9.230334   -3.2094162  -2.7193034
  -10.870816 ]]
[[-7.593111   -0.7545757   4.078487    5.8787546  -3.714365   -0.80454606
  -4.224027  ]]
[[ -0.39514235  12.55944     11.599643   -13.702196    -0.09588697
    2.5890431   -4.004258  ]]
[[ -4.0715575  -1.7448945  16.440489  -11.551546   -5.0167994   2.4815123
    3.002321 ]]
[[ -1.5920923    5.1290464   16.903189   -11.630041     0.03243659
   -1.6330348   -1.7303469 ]]
[[ -0.1133173    0.38048935  14.876246   -15.137547    -4.8216977
    4.8773346    1.0445335 ]]
[[ -2.301157    -3.7539954   26.580938   -10.650669    -1.6064768
   -3.768481    -0.92555016]]
[[  7.681019   13.654005  -15.821719  -18.706589    2.5101466   4.3828535
   

   1.6629503]]
[[  6.3090105  12.724709    4.6746573 -19.545591    3.9631793  -2.7281044
    0.8691824]]
[[  2.7112918 -29.791883    2.051133   -4.8127604  11.799652   10.590304
    2.495526 ]]
[[ 7.76611    -4.7853875  10.012254   -3.8578994  -3.2883427   0.39148885
  -6.4383583 ]]
[[  6.7323985 -37.201973   13.602769  -17.62227     4.2201133  17.488401
   11.835021 ]]
[[  6.9612584 -13.190753   12.28476    -5.373423    3.2838218   0.636792
   -4.4669204]]
[[  2.9150817 -50.22064    28.421158  -18.009268    2.4966953  13.269232
   19.7752   ]]
[[  5.1877136   -7.089605   -25.129457     7.8536515    9.534325
    3.421363    -0.30187246]]
[[ -1.7047226 -36.586147  -15.13572    -6.2639685  16.675856   10.2571125
   19.900423 ]]
[[  6.7998376 -12.1388235 -34.030983    3.9915526  23.422302    7.9654737
   -0.7621843]]
[[  0.95033187 -31.914654    -0.60174096  -4.666693    -8.809365
   17.81318     21.447384  ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 

[[-0.05180804  2.7028742   9.568742   -9.205665   -2.7667737   3.3884785
  -4.59295   ]]
[[-4.1691122e+00 -1.1114591e-02  2.4735308e+01 -1.4268078e+01
  -2.0544493e+00 -1.2739533e-01  1.7482259e+00]]
[[  8.614355    17.137272   -21.348764   -13.647547     4.502732
    2.8668563   -0.55673236]]
[[-4.1477776  6.8613105 -7.983989  -9.130894   5.4317822  3.531799
   1.7902753]]
[[  6.8727727  24.57605   -31.28831   -10.564665    7.2951045  -2.0188367
    1.4919317]]
[[  0.81556004  13.064654   -14.231362    -9.625822     4.848226
    2.3490615   -0.31855127]]
[[ 15.840543  -19.78444   -20.570547   -7.5837107  11.621647   15.552415
    2.541991 ]]
[[  0.7065117   3.5669875 -20.586363   10.904323    6.8276668   0.9753507
   -9.342494 ]]
[[ 11.696225  -18.4416    -15.251716   -1.3916243   4.542359   14.739581
   -4.3282995]]
[[ -5.410326   -3.1151357 -21.759317   50.500263   -5.7972608 -16.408396
  -29.605602 ]]
[[  2.076134  -22.963745  -21.651684   26.954016    1.6329677   5.6390452
   -9.6

[[  0.9187042  -0.6594019  19.294622  -13.736926    2.992442   -3.001314
   -4.900528 ]]
[[  7.4989176  12.657895  -17.481224  -12.27758     1.228303    4.2140613
    6.216204 ]]
[[ -0.62313825  27.114958   -31.037302    -7.734021    11.81202
   -2.7016554   -5.5134397 ]]
[[ 13.100036    9.438278  -21.117075  -13.461747    1.4669435   4.1663346
    8.299445 ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[ -6.2004       0.65974045   0.7913815   18.833784    -5.383792
   -4.548825   -10.175612  ]]
[[-7.3466206   1.6244636   4.1721807   6.50288    -6.2807155  -0.81039494
  -6.9203773 ]]
[[ -3.8469706   5.871923    2.019808    9.230334   -3.2094162  -2.7193034
  -10.870816 ]]
[[-7.593111   -0.7545757   4.078487    5.8787546  -3.714365   -0.80454606
  -4.224027  ]]
[[ -0.39514235  12.55944     11.599643   -13.702196    -0.09588697
    2.5890431   -4.004258  ]]
[[ -4.0715575  -1.7448945  16.440489  -11.551546   -5.0167994   2.4815123
    3.002321 ]]
[[ 

[[  7.2284384  -5.92007    16.630314    0.1076194  -7.234699   -1.0887691
  -15.092019 ]]
[[  6.2209153 -38.870457   19.366833  -15.870121    1.3911273  15.581693
   14.074201 ]]
[[  7.978386  -10.816437   22.303034   -0.7771133  -4.8917875  -3.1805017
  -14.579272 ]]
[[ 11.179942 -28.089233 -14.95244  -15.718321  10.702777  19.366068
   18.308397]]
[[  4.8112464   -0.39618826 -16.641262    13.02747      3.4247057
    0.60644656 -17.069733  ]]
[[  5.3389053 -31.583235  -11.639089  -20.02246     9.716182   20.719597
   22.277716 ]]
[[  2.0560935 -10.431684  -18.169966   49.405083  -10.482335   -9.174219
  -35.043297 ]]
[[ -2.4870436 -35.7992    -19.6681      7.4922485   3.1395395  15.779296
   15.3896675]]
[[-23.479336 -42.269363 -22.221117 133.2179   -37.164146 -21.769028
  -54.90806 ]]
[[-12.301264  -33.64299    -5.74468     6.0828967   4.602159   -7.47858
   18.745497 ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[-7.677657    0.20305586  2.371

...

[[-7.4423275  0.288414   8.510629  -6.152202   1.5159714 -9.0308695
  -5.4351435]]
[[ -5.302621   -1.0102992   9.0573225 -12.055856    4.074174   -7.3732758
   -9.094865 ]]
[[ -1.8760743   7.4858227 -18.195747  -10.534017    7.710505   -2.784848
   -7.8195972]]
[[ -2.459073   14.924334  -16.345318   -8.006405   -1.2522762   2.7310383
   -6.198517 ]]
[[ -1.4005859  10.361507  -14.462359   -6.513119   -2.0777853   2.348416
   -6.0151105]]
[[ -5.2102456    8.430425   -28.381514     7.4014273    6.172655
    0.35294467 -15.406537  ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[ -7.5465803    0.09281803  -0.8589312   18.314896    -2.776148
   -3.1689253  -10.292765  ]]
[[ 0.644005   3.5993927  4.617439  -1.8803748 -1.819444   1.0670968
  -3.229931 ]]
[[ 15.133484     4.536998     0.23797189 -10.459449    -2.6149304
   -3.2636693   -2.3082592 ]]
[[-9.787315  -7.0993104  7.2129536  7.2669587  1.0264452 -7.8966184
  -8.527196 ]]
[[-12.984919    -0.523629

[[-3.9519374   2.7574606   4.1798234   8.836837   -0.99679065 -5.74975
  -6.153217  ]]
[[ 0.8444511  7.7899003  2.700349  -6.7680817  2.5689247 -2.2078216
  -4.000022 ]]
[[ 14.046126   -1.3214526  10.086638  -14.8906765   0.6958792  -3.7473783
   -3.206303 ]]
[[-1.4072050e+01 -4.4936662e+00  1.5858126e+01  1.3933782e-02
  -4.1106871e-01 -9.0290117e+00 -7.0533900e+00]]
[[-7.4423275  0.288414   8.510629  -6.152202   1.5159714 -9.0308695
  -5.4351435]]
[[ -5.302621   -1.0102992   9.0573225 -12.055856    4.074174   -7.3732758
   -9.094865 ]]
[[ -1.8760743   7.4858227 -18.195747  -10.534017    7.710505   -2.784848
   -7.8195972]]
[[ -2.459073   14.924334  -16.345318   -8.006405   -1.2522762   2.7310383
   -6.198517 ]]
[[ -1.4005859  10.361507  -14.462359   -6.513119   -2.0777853   2.348416
   -6.0151105]]
[[ -5.2102456    8.430425   -28.381514     7.4014273    6.172655
    0.35294467 -15.406537  ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[-4.627763

[[-4.3582077  1.1399302  5.689564   7.700468  -4.3978467 -0.3558969
  -7.154077 ]]
[[ 0.28974077  5.563727    2.5887382  -5.5505576  -0.86388516  1.5204605
  -3.2553923 ]]
[[ 13.65593    -4.6833367  10.415803  -12.775658   -2.2551167  -0.7372114
   -2.1859028]]
[[-15.092936   -7.529778   17.575516    1.5830327  -5.022824   -6.072705
   -4.501218 ]]
[[-9.028265   -2.260898   11.126365   -0.41849554 -3.840647   -6.193764
  -8.728091  ]]
[[ -6.4158134  -2.8402562  10.685699   -5.8513722  -1.0617702  -4.8054605
  -12.681989 ]]
[[ -3.1138747   5.5025053 -16.294226   -3.3459623   2.6356518  -0.7689039
  -11.862925 ]]
[[ -1.7649834   2.2230387 -13.898665   -1.8374528   1.0529954  -1.1840068
   -9.5773   ]]
[[ -1.6997225 -17.759768  -15.113312   18.012342    5.283108   -7.1850476
  -11.985211 ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[-7.221196  -1.2534282  3.9018965 17.187077  -0.1641896 -9.029792
  -6.151119 ]]
[[-3.9519374   2.7574606   4.1798234 

[[ -6.7021084 -10.818417  -18.096792   23.220528    1.4528267 -11.106323
  -11.45002  ]]
[[ -3.6352062   -9.37521    -18.783895    13.173504    -0.38169697
   -4.0114207  -15.348873  ]]
[[ -1.3311708  -3.85385    -7.705053  -11.06677     5.6427417   0.5768493
   -8.839268 ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[-4.6277637  3.51684    5.1682434 13.867216  -4.227979  -7.6714635
  -7.6535063]]
[[ 9.883214    3.5716467  11.043118   -8.89209    -6.804193   -4.97975
  -0.91009444]]
[[12.304852   1.5203023  5.867313  -7.4032817 -6.3123107 -5.7634897
  -1.6942456]]
[[-16.388084     0.41138792  11.59625      8.322246    -9.515168
  -10.692264    -6.553542  ]]
[[-12.552564    1.4377623   8.602594    1.888394   -3.8752244  -9.601165
   -7.287343 ]]
[[-10.20664      9.987902   -17.435091     6.46553     -0.43745115
   -7.581289    -7.381199  ]]
[[ -9.522907    9.057763  -16.939356    8.871872   -1.7157056  -8.200014
   -7.704551 ]]
[[ -6.7021084 -10.8

[[ 0.28974077  5.563727    2.5887382  -5.5505576  -0.86388516  1.5204605
  -3.2553923 ]]
[[ 13.65593    -4.6833367  10.415803  -12.775658   -2.2551167  -0.7372114
   -2.1859028]]
[[-15.092936   -7.529778   17.575516    1.5830327  -5.022824   -6.072705
   -4.501218 ]]
[[-9.028265   -2.260898   11.126365   -0.41849554 -3.840647   -6.193764
  -8.728091  ]]
[[ -6.4158134  -2.8402562  10.685699   -5.8513722  -1.0617702  -4.8054605
  -12.681989 ]]
[[ -3.1138747   5.5025053 -16.294226   -3.3459623   2.6356518  -0.7689039
  -11.862925 ]]
[[ -1.7649834   2.2230387 -13.898665   -1.8374528   1.0529954  -1.1840068
   -9.5773   ]]
[[ -1.6997225 -17.759768  -15.113312   18.012342    5.283108   -7.1850476
  -11.985211 ]]
[[-6.3235283 -2.3726945  2.4884126 17.7358    -4.233413  -4.780684
  -8.358526 ]]
[[-6.112264 -6.541     8.28583  19.67451  -5.209121 -5.420482 -9.99055 ]]
[[ 2.9702604 -3.487516  12.255828  -1.9787248 -3.0209548 -0.9772126
  -1.4770648]]
[[ 5.844468    0.45634794  4.428467    0.1008

In [8]:
import tensorflow_probability as tfp
tfd = tfp.distributions
def test():
    dist = tfd.Multinomial(total_count=1., probs=[.5, .4, .1, 0.0])

    with tf.Session() as sess:

        g = dist.sample(10)
        results = sess.run([g])

    for r in results:
        print(r)

In [9]:
test()

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
